<a href="https://colab.research.google.com/github/miftahulmuhaemen/Pothole-Object-Detection-TF-API-/blob/master/TF_Object_Detection_API_(Pothole)_(GPU)_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU AVAILABLITY
> Finding out the avalability of the GPU Support before training the Model on Colab.

In [0]:
# !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
# !pip install gputil
# !pip install psutil
# !pip install humanize
# import psutil
# import humanize
# import os
# import GPUtil as GPU
# GPUs = GPU.getGPUs()
# gpu = GPUs[0]
# def printm():
#  process = psutil.Process(os.getpid())
#  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
#  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm() 

# SETUP

Getting authentication from GCP and GDRIVE.
> GCP for its Storage and Ai-Platform (Default-Disabled)


In [0]:
# !gcloud auth login
# !gcloud auth application-default login

> Gdrive for local drive, sometimes some command only accept local, which is impossible for gs:// path, then again Google Colab has limited time use (12 hours) and everything would be erased after that, so to maintain more easier uses we mount it and use it as 'eternal local drive'.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Installing necessary library to run and copying Tensorflow Object Detection API from its repository.

In [0]:
%%capture
!apt-get install -y -qq protobuf-compiler python-pil python-lxml
!pip install tensorboardcolab absl-py
!git clone --quiet https://github.com/tensorflow/models.git

Adding path.

In [0]:
import os
import sys
os.chdir('models/research')
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!protoc object_detection/protos/*.proto --python_out=.
sys.path.append('/content/models/research/slim')

Test the system if it is ready to train the model.

In [0]:
%%capture
!python object_detection/builders/model_builder_test.py

Extracting the necessary library.

In [0]:
%%capture
!chmod u+rwx object_detection/dataset_tools/create_pycocotools_package.sh 
!object_detection/dataset_tools/create_pycocotools_package.sh /tmp/pycocotools
!python setup.py sdist
!(cd slim && python setup.py sdist)

Adding the project name for using the Google Storage.

In [0]:
# %%capture
# !gcloud config set project robust-app-devs

# PREPARE DATASET

Put the training and evaluation dataset to each directory in Google Storage/Google Drive, and Tensorflow Object Detection API only accept TFRecord (.tfrecord/.record).

Then download and configure the pipeline config each model dataset path or the checkpoint path as well.

*   Total data : 1793 tfrecords (Frames) [16 Files]
*   Image Augmentation (On-Fly) : Random Horizontal Flip, Crop, Saturation, Hue, Contrast, Brightness and Grayscale.
*   Image Resizer : 480x480 (Squeezed) ~ 228









In [0]:
bucket_name = 'tf_cnn_pothole_drone_4803018/data' #@param {type:"string"}
fps = '15FPS' #@param ["1FPS","15FPS","30FPS"]
ratio = '6040' #@param ["9010","8020","6040"]
# augmentation = '_FALSEAUGMENTATION' #@param ["_FALSEAUGMENTATION",""]
PretrainedModel = 'ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03' #@param ["ssd_mobilenet_v1_coco_2018_01_28", "ssd_mobilenet_v2_coco_2018_03_29", "ssd_mobilenet_v3_large_coco_2019_08_14","ssd_mobilenet_v3_small_coco_2019_08_14", "ssd_inception_v2_coco_2018_01_28", "ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03"]

# TRAINING ON GCP

If only you want to train it on Ai-Platform, or you could test your training on local (Colab) first, then if it lack the power you want, switch it to this.

In [0]:
# !gcloud ai-platform  jobs submit training {PretrainedModel}_{fps}_{ratio} \
# --runtime-version 1.12 --job-dir=gs://tf_cnn_pothole_drone_4803018/ModelTraining/model/{PretrainedModel}_{fps}_{ratio} \
# --packages dist/object_detection-0.1.tar.gz,slim/dist/slim-0.1.tar.gz,/content/drive/'My Drive'/pycocotools-2.0.tar.gz \
# --module-name object_detection.model_main \
# --region us-central1 \
# --config /content/drive/'My Drive'/cloud.yml \
# -- \
# --model_dir=gs://tf_cnn_pothole_drone_4803018/ModelTraining/model/{PretrainedModel}_{fps}_{ratio} \
# --pipeline_config_path=gs://tf_cnn_pothole_drone_4803018/ModelTraining/config/data_{PretrainedModel}_pipeline_{fps}_{ratio}.config

In [0]:
# !gcloud ai-platform jobs stream-logs object_detection_training_08112019_colab_v3

# TRAINING ON GOOGLE COLABORATORY (LOCAL)

   Failed, due to various reason, mainly Out of Memory(OOM) and NaN Loss. OOM triggered because this Colab using P4/K80/P100 (1x), so the solution could be the GCP, utilizing more RAM packed with its GPU, but
   this solution is costly. It's even more worthless if the result is NaN Loss where the solution is to increasing the batch number or reducing learning rate, where lead us to OOM again, hell loop. Here is the failed list,
    # data_ssd_mobilenet_v2_coco_2018_03_29_pipeline.config \
    # data_ssd_mobilenet_v2_quantized_300x300_coco_2019_01_03_pipeline.config \
    # data_faster_rcnn_resnet101_coco_2018_01_28_pipeline.config \
    # data_faster_rcnn_resnet50_coco_2018_01_28_pipeline.config \
    # data_rfcn_resnet101_coco_2018_01_28_pipeline.config \
    # data_faster_rcnn_inception_v2_coco_2018_01_28_pipeline.config \

   Worked one,
    # data_ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03_pipeline.config \
    # data_ssd_inception_v2_coco_2018_01_28_pipeline.config \
    # data_ssd_mobilenet_v1_coco_2018_01_28_pipeline.config \
    # data_ssd_mobilenet_v2_coco_2018_03_29_pipeline.config \
    # data_ssd_mobilenet_v3_large_coco_2019_08_14_pipeline.config \

> Google Drive

In [0]:
!python object_detection/model_main.py \
    --pipeline_config_path=/content/drive/'My Drive'/ModelTraining/config/data_{PretrainedModel}_pipeline_{fps}_{ratio}.config \
    --model_dir=/content/drive/'My Drive'/ModelTraining/model/{PretrainedModel}_{fps}_{ratio} \
    --alsologtostderr

> Google Cloud Storage

In [0]:
# !python object_detection/model_main.py \
#     --pipeline_config_path=gs://{bucket_name}/data_{PretrainedModel}_pipeline_{fps}_{ratio}.config\
#     --model_dir=gs://{bucket_name}/model/{PretrainedModel}_{fps}_{ratio} \
#     --alsologtostderr    

# WATCH THE PROGRESS ON TENSORBOARD

model_output refer to model_dir/job_dir where the output model is saved

> Google Cloud Tensorboard (Cloud Shell)

tensorboard --logdir=gs://model_output --port=8080

> Collaboratory Tensorboard

In [0]:
%load_ext tensorboard
import tensorflow as tf
import datetime, os
from tensorboard import notebook
%tensorboard --logdir /content/drive/'My Drive'/ModelTraining/model/{PretrainedModel}_{fps}_{ratio}/

# EXPORTING CHECKPOINT 


In order to test it on real data, we need to export the checkpoint first.

Check the checkpoint number, model.ckpt--{number_checkpoint},
> Google Cloud Storage

In [0]:
# !gsutil ls gs://{bucket_name}/model/{PretrainedModel}_{fps}_{ratio}

> Google Drive

In [0]:
!ls /content/drive/'My Drive'/ModelTraining/model/{PretrainedModel}_{fps}_{ratio}

In [0]:
Checkpoint = 30000 #@param {type:"number"}

Now export it,

> Google Cloud Storage

In [0]:
# !python object_detection/export_inference_graph.py \
# --input_type image_tensor \
# --trained_checkpoint_prefix gs://{bucket_name}/model/{PretrainedModel}_{fps}_{ratio}/model.ckpt-{Checkpoint} \
# --pipeline_config_path gs://{bucket_name}/data_{PretrainedModel}_{fps}_{ratio}_pipeline.config \
# --output_directory gs://{bucket_name}/export/{PretrainedModel}_{fps}_{ratio}

> Google Drive

In [0]:
# !python object_detection/export_inference_graph.py \
# --input_type=image_tensor \
# --trained_checkpoint_prefix=/content/drive/'My Drive'/ModelTraining/model/{PretrainedModel}_{fps}_{ratio}/model.ckpt-{Checkpoint} \
# --pipeline_config_path=/content/drive/'My Drive'/ModelTraining/config/data_{PretrainedModel}_pipeline_{fps}_{ratio}.config \
# --output_directory=/content/drive/'My Drive'/ModelTraining/export/{PretrainedModel}_{fps}_{ratio}

# TEST AGAINST REAL DATA


In [0]:
%%capture
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import time
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline
sys.path.append("..")
from object_detection.utils  import label_map_util
from object_detection.utils  import visualization_utils as vis_util

In [0]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # you should return the final output (image with lines are drawn on lanes)
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            image_process = detect_objects(image, sess, detection_graph)
            return image_process

In [0]:
def detect_objects(image_np, sess, detection_graph):
    # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

    # Each box represents a part of the image where a particular object was detected.
    boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

    # Each score represent how level of confidence for each of the objects.
    # Score is shown on the result image, together with the class label.
    scores = detection_graph.get_tensor_by_name('detection_scores:0')
    classes = detection_graph.get_tensor_by_name('detection_classes:0')
    num_detections = detection_graph.get_tensor_by_name('num_detections:0')

    # Actual detection.
    (boxes, scores, classes, num_detections) = sess.run(
        [boxes, scores, classes, num_detections],
        feed_dict={image_tensor: image_np_expanded})

    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=8)
    return image_np

Video feed path still hardcoded, so you need to define it yourself.
And acces the output from Google Drive.

In [0]:
test = ["DJI_0033","DJI_0035", "DJI_0043","DJI_0046"]
models = ["ssd_mobilenet_v2_coco_2018_03_29","ssd_mobilenet_v3_large_coco_2019_08_14","ssd_resnet50_v1_fpn_shared_box_predictor_640x640_coco14_sync_2018_07_03"]
frames = ["30FPS","15FPS","1FPS"]
splits = ["9010","8020","6040"]

for video_name in test:
  for model in models :
    for frame in frames :
      for split in splits :

        # Path to frozen detection graph. This is the actual model that is used for the object detection.
        # Adding path to the exported model and its label, also the class number.
        PATH_TO_CKPT = '/content/drive/My Drive/ModelTraining/export/'+model+'_'+frame+'_'+split+'/frozen_inference_graph.pb'
        PATH_TO_LABELS = os.path.join('/content/drive/My Drive/ExportedModel', 'tf_label_map.pbtxt')
        NUM_CLASSES = 1

        # Parsing Model
        detection_graph = tf.Graph()
        with detection_graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')

        # Parsing Label
        label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
        categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
        category_index = label_map_util.create_category_index(categories)

        clip1 = VideoFileClip("/content/drive/My Drive/RAW_FOOTAGE_BJB_POTHOLE/"+video_name+".MOV")
        white_output = "/content/drive/My Drive/ExportedModel/exported_videos/LastCheckpoint/"+model+"_"+frame+"_"+split+"_"+video_name+".mp4" #Make sure it's the same path as the line below.
        exist = !find /content/drive/'My Drive'/ExportedModel/exported_videos/LastCheckpoint/{model}_{frame}_{split}_{video_name}.mp4  -type f | wc -l #To prevent overwriting the same file
        if exist == ['1'] :
          pass
        else :
          white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
          white_clip.write_videofile(white_output, audio=False)

# SOURCE




> https://towardsdatascience.com/how-to-train-your-own-object-detector-with-tensorflows-object-detector-api-bec72ecfe1d9

---


> https://averdones.github.io/tensorflow-object-detection-star-wars/

---


> https://github.com/averdones/star_wars_object_detection/blob/master/object_detection_sw.ipynb

---


> https://cloud.google.com/blog/products/gcp/training-an-object-detector-using-cloud-machine-learning-engine

---


> https://github.com/tensorflow/models/tree/master/research/object_detection